# Move Mookodi FITS files to subdirectories prior to running pipeline
The TEA-Phot.py pipeline assumes that flats, biases and science images are all in separate directories.  This code creates subdirectories (sub-folders) of the current folder, and using the contents of the headers, automacitally moves the fits files into separate folders, in preparation for running TEA-phot.py.<br>
<br>
First we create some sub-folders.  Note that on a Mac, folders start with a "/", while I am fairly sure on a Windows machine they start with a "\" .  Hence we have to use the Pathlib library in order to get the path. This library will automatically use "\\" on Windows computers, and "/" on Macs. I have not tested this on a Windows computer, and just trusting the documentation for Pathlib.

In [ ]:
import os
import shutil
import glob
from pathlib import Path
from astropy.io import fits

#create a directory
def create_dir(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

######################################################################################

# move a file to a specific subdirectory
def move_file(file,new_dir):
    shutil.move(file, new_dir)
    print('Moved:', file)


####################################################################################


#main program starts here

my_cwd = Path(os.getcwd())
print ("The current working directory is %s" % my_cwd)

#bias_path = my_cwd + sub_fold_str + "Biases" 
bias_path = my_cwd / "Biases"
print(bias_path)


#hardwiring in the names of the sub-folders
#hardwiring in g and i filters, but we could make this more general in the future
gflat_path = my_cwd / "g_flats"
iflat_path = my_cwd / "i_flats"
gscience_path = my_cwd / "g_science"
iscience_path = my_cwd / "i_science"


#Create the new folders 
create_dir(bias_path)
create_dir(gflat_path)
create_dir(iflat_path)
create_dir(gscience_path)
create_dir(iscience_path)


## Loop through all fits files in the current folder and move to the appropriate sub-folder
Note that we read the type of fits file from the fits header, and separate out the images taken with the g and i filters into separate folders as they will need to be processed separately with TEA-phot.py

In [ ]:


# put the names of all of the fits files into a list 

fits_files = list(my_cwd.glob('*.fits'))


for file in fits_files:
    
    hdul = fits.open(file)  # open a FITS file
    hdr = hdul[0].header  #extract the header information 
    #print(file)
    #print(hdr)

    
    bias_keyword = "BIAS"    #this might need to be changed for SHOC
    flat_keyword = "SKYFLAT" #this might need to be changed for SHOC
    
#    flat_keyword = "Fermi-GBM_Test_Positio_Mon_30_Jan_23_0344_108_-35"   #this was my test
    
#    print("Filter:", hdr['FILTERA'])
    
    # extract file name from the file path
    file_name = os.path.basename(file)
        
    if hdr['OBJECT'] == bias_keyword :
        move_file(file_name,bias_path)
    elif hdr['OBJECT'] == flat_keyword :
        if hdr['FILTERA'] == 'g' :
            move_file(file_name,gflat_path)
        elif hdr['FILTERA'] == 'i' :
            move_file(file_name,iflat_path)
        else :
            print("File is not a g or i filter: ", file_name,  hdr['FILTERA'] )
    else :
     #must be a science image
        if hdr['FILTERA'] == 'g' :
            move_file(file_name,gscience_path)
        elif hdr['FILTERA'] == 'i' :
            move_file(file_name,iscience_path)
        else :
            print("File is not a g or i filter: ",  file_name, hdr['FILTERA'])


print("Finished moving files")
